### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.11.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.11.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.11.1-alpha"
#r "nuget: Microsoft.Bing.Search.WebSearch, 1.0.0"

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.Bing.WebSearch;
using System.ComponentModel;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
var bingSearchAPIKey = config["BingSearch:APIKey"];

Installed Packages Microsoft.Bing.Search.WebSearch, 1.0.0 Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0 Microsoft.SemanticKernel, 1.11.1 Microsoft.SemanticKernel.Plugins.Core, 1.11.1-alpha Microsoft.SemanticKernel.Plugins.Web, 1.11.1-alpha

In [2]:
var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: azureOpenAIModelDeploymentName,
        endpoint: azureOpenAIEndpoint,
        apiKey: azureOpenAIAPIKey)
    .Build();

In [3]:
public class WebSearchResult
{
    public int Id { get; set; } = 0;
    public string Name { get; set; } = string.Empty;
    public string Snippet { get; set; } = string.Empty;
    public string Url { get; set; } = string.Empty;
}

public class WebSearchResultsPlugin
{
    [KernelFunction]
    [Description("Retrieves the web search results with identified URL sources for a given baseball player.")]
    public async Task<string> GetWebSearchResults(string nameOfBaseballPlayer)
    {
        var configurationBuilder = new ConfigurationBuilder()
            .SetBasePath(Directory.GetCurrentDirectory())
            .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
            .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
        var config = configurationBuilder.Build();
        var bingSearchAPIKey = config["BingSearch:APIKey"];

        var webSearchResultsString = "Web search results:\r\n\r\n";
        var footNotes = string.Empty;
        var bingSearchId = 0;
        var webSearchResults = new List<WebSearchResult>();

        var bingSearchClient = new WebSearchClient(new ApiKeyServiceClientCredentials(bingSearchAPIKey));
        var bingWebData = await bingSearchClient.Web.SearchAsync(query: "baseball hall of fame " + nameOfBaseballPlayer, count: 8);

        if (bingWebData?.WebPages?.Value?.Count > 0)
        {
            // Itertate over the Bing Web Pages (Non-Cache Results)
            foreach (var bingWebPage in bingWebData.WebPages.Value)
            {
                bingSearchId++;

                webSearchResultsString += string.Format("[{0}]: \"{1}: {2}\"\r\nURL: {3}\r\n\r\n",
                    bingSearchId, bingWebPage.Name, bingWebPage.Snippet, bingWebPage.Url);

                footNotes += string.Format("[{0}]: {1}: {2}  \r\n",
                    bingSearchId, bingWebPage.Name, bingWebPage.Url);

                webSearchResults.Add(new WebSearchResult
                {
                    Id = bingSearchId,
                    Name = bingWebPage.Name,
                    Snippet = bingWebPage.Snippet,
                    Url = bingWebPage.Url
                });
            }
        }

        return webSearchResultsString;
    }
}
semanticKernel.ImportPluginFromType<WebSearchResultsPlugin>();

In [13]:
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the hall of fame? 
Provide sources of information and explain your reasoning.
Ensure to cite all the sources using [number] notation of each URL after the reference in order.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

Based on the available information, I would recommend that Mike Trout should be considered for the Hall of Fame. Here’s the reasoning:

1. **Eligibility and Qualifications**: Mike Trout has satisfied the Hall of Fame election eligibility rule by playing in ten Major League championship seasons [1]. His career achievements, including being an 11-time MLB All-Star and a three-time American League Most Valuable Player, further solidify his qualifications [5].

2. **Performance Metrics**: Despite injuries that have impacted his playtime in recent years [2][3][8], Trout's career statistics remain impressive. His performance metrics, such as MVP awards and All-Star appearances, compare favorably against many current Hall of Famers [6].

3. **Injury Considerations**: Although injuries have caused him to miss significant games, it's important to consider the overall impact and quality of his play when he is active. His injuries, while a factor in evaluating his career, should not overshadow hi